In [1]:
from src.binaryClassificationModel import BinaryClassificationModel
import pickle
import torch

from src.evaluation import Evaluation
from sklearn.model_selection import train_test_split

with open('../data/processed/preprocessed_trainSet.pkl', 'rb') as f:
    params = pickle.load(f)
X_train, X_test, y_train, y_test = train_test_split(params['X'], params['y'], test_size=0.2, random_state=0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(torch.version.cuda)

Using device: cpu
None


In [2]:
model = BinaryClassificationModel(X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model = model.to(device)

from torch.utils.data import DataLoader

dataset = list(zip(X_train, y_train))
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

for epoch in range(30):
    model.train()
    total_loss = 0

    for X_batch, y_batch in dataloader:
        y_batch = y_batch.unsqueeze(1)
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = model.loss(logits, y_batch)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")


Epoch 1, Loss: 0.4852
Epoch 2, Loss: 0.2334
Epoch 3, Loss: 0.1280
Epoch 4, Loss: 0.0944
Epoch 5, Loss: 0.0815
Epoch 6, Loss: 0.0742
Epoch 7, Loss: 0.0699
Epoch 8, Loss: 0.0668
Epoch 9, Loss: 0.0640
Epoch 10, Loss: 0.0621
Epoch 11, Loss: 0.0603
Epoch 12, Loss: 0.0590
Epoch 13, Loss: 0.0579
Epoch 14, Loss: 0.0565
Epoch 15, Loss: 0.0557
Epoch 16, Loss: 0.0543
Epoch 17, Loss: 0.0537
Epoch 18, Loss: 0.0529
Epoch 19, Loss: 0.0521
Epoch 20, Loss: 0.0511
Epoch 21, Loss: 0.0505
Epoch 22, Loss: 0.0499
Epoch 23, Loss: 0.0495
Epoch 24, Loss: 0.0488
Epoch 25, Loss: 0.0482
Epoch 26, Loss: 0.0474
Epoch 27, Loss: 0.0471
Epoch 28, Loss: 0.0466
Epoch 29, Loss: 0.0457
Epoch 30, Loss: 0.0455


In [3]:
test_set = list(zip(X_test, y_test))
test_loader = DataLoader(test_set, batch_size=10, shuffle=True)

model.eval()
test_loss = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_batch = y_batch.unsqueeze(1)
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        logits = model(X_batch)
        loss = model.loss(logits, y_batch)
        test_loss += loss.item()

print(f"Test Loss: {test_loss/len(test_loader):.4f}")

Test Loss: 0.0735


In [4]:
torch.save(model.state_dict(), "../models/BinaryClassificationModel.pth")